In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
from numpy.fft import fft

from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings('ignore')

plt.rcParams["figure.figsize"] = (30, 20)

## process_file() takes a string with a path to the .csv
def process_file(pathname):
    res = pd.read_csv(pathname, sep = ',', header = 0, index_col = None, names = ['time', 'seconds', 'z', 'y', 'x'])
    return res

In [45]:
## Importing Data

## Walking
d_rp5min = process_file('processed_data/drp_5min_bw.csv')
d_rp5min['movement'] = 'walking'
d_walk_split = np.array_split(d_rp5min, 120)

m_rp5min = process_file('processed_data/mrp_5min_bw.csv')
m_rp5min['movement'] = 'walking'
m_walk_split = np.array_split(m_rp5min, 120)

s_rp5min = process_file('processed_data/srp_5min_bw.csv')
s_rp5min['movement'] = 'walking'
s_walk_split = np.array_split(s_rp5min, 120)

## Staircase
d_up1 = process_file('processed_data/d_up_1_bw.csv')
d_up2 = process_file('processed_data/d_up_2_bw.csv')
d_up3 = process_file('processed_data/d_up_3_bw.csv')
d_up4 = process_file('processed_data/d_up_4_bw.csv')
d_up5 = process_file('processed_data/dlp_upstairs_1_bw.csv')
d_up6 = process_file('processed_data/dlp_upstairs_2_bw.csv')

d_down1 = process_file('processed_data/d_down_1_bw.csv')
d_down2 = process_file('processed_data/d_down_2_bw.csv')
d_down3 = process_file('processed_data/d_down_3_bw.csv')
d_down4 = process_file('processed_data/d_down_4_bw.csv')
d_down5 = process_file('processed_data/dlp_downstairs_1_bw.csv')
d_down6 = process_file('processed_data/dlp_downstairs_2_bw.csv')

m_up24_1_bw = process_file('processed_data\m_up24_1_bw.csv')
m_up24_2_bw = process_file('processed_data\m_up24_2_bw.csv')
m_up24_3_bw = process_file('processed_data\m_up24_3_bw.csv')
m_up24_4_bw = process_file('processed_data\m_up24_4_bw.csv')
m_up24_5_bw = process_file('processed_data\m_up24_5_bw.csv')

m_down24_1_bw = process_file('processed_data\m_down24_1_bw.csv')
m_down24_2_bw = process_file('processed_data\m_down24_2_bw.csv')
m_down24_3_bw = process_file('processed_data\m_down24_3_bw.csv')
m_down24_4_bw = process_file('processed_data\m_down24_4_bw.csv')
m_down24_5_bw = process_file('processed_data\m_down24_5_bw.csv')

d_up1['movement'] = 'upstairs'
d_up2['movement'] = 'upstairs'
d_up3['movement'] = 'upstairs'
d_up4['movement'] = 'upstairs'
d_up5['movement'] = 'upstairs'
d_up6['movement'] = 'upstairs'

d_down1['movement'] = 'downstairs'
d_down2['movement'] = 'downstairs'
d_down3['movement'] = 'downstairs'
d_down4['movement'] = 'downstairs'
d_down5['movement'] = 'downstairs'
d_down6['movement'] = 'downstairs'

m_up24_1_bw['movement'] = 'upstairs'
m_up24_2_bw['movement'] = 'upstairs'
m_up24_3_bw['movement'] = 'upstairs'
m_up24_4_bw['movement'] = 'upstairs'
m_up24_5_bw['movement'] = 'upstairs'

m_down24_1_bw['movement'] = 'downstairs'
m_down24_2_bw['movement'] = 'downstairs'
m_down24_3_bw['movement'] = 'downstairs'
m_down24_4_bw['movement'] = 'downstairs'
m_down24_5_bw['movement'] = 'downstairs'

d_up1_split = np.array_split(d_up1, 4)
d_up2_split = np.array_split(d_up2, 4)
d_up3_split = np.array_split(d_up3, 4)
d_up4_split = np.array_split(d_up4, 4)
d_up5_split = np.array_split(d_up5, 4)
d_up6_split = np.array_split(d_up6, 4)

d_down1_split = np.array_split(d_down1, 4)
d_down2_split = np.array_split(d_down2, 4)
d_down3_split = np.array_split(d_down3, 4)
d_down4_split = np.array_split(d_down4, 4)
d_down5_split = np.array_split(d_down5, 4)
d_down6_split = np.array_split(d_down6, 4)

m_up24_1_split = np.array_split(m_up24_1_bw, 4)
m_up24_2_split = np.array_split(m_up24_2_bw, 4)
m_up24_3_split = np.array_split(m_up24_3_bw, 4)
m_up24_4_split = np.array_split(m_up24_4_bw, 4)
m_up24_5_split = np.array_split(m_up24_5_bw, 4)

m_down24_1_split = np.array_split(m_down24_1_bw, 4)
m_down24_2_split = np.array_split(m_down24_2_bw, 4)
m_down24_3_split = np.array_split(m_down24_3_bw, 4)
m_down24_4_split = np.array_split(m_down24_4_bw, 4)
m_down24_5_split = np.array_split(m_down24_5_bw, 4)

tmove = []
tdata = []

for i in [d_walk_split, m_walk_split, s_walk_split,
            d_up1_split, d_up2_split, d_up3_split, d_up4_split, d_up5_split, d_up6_split, d_down1_split, d_down2_split, d_down3_split, d_down4_split, d_down5_split, d_down6_split,
            m_up24_1_split, m_up24_2_split, m_up24_3_split, m_up24_4_split, m_up24_5_split, m_down24_1_split, m_down24_2_split, m_down24_3_split, m_down24_4_split, m_down24_5_split]:
    for j in range(len(i) - 1):
        temp = i[j]

        if ((temp['movement'] == 'walking').all()):
            tmove.append('walking')
        elif ((temp['movement'] == 'upstairs').all()):
            tmove.append('upstairs')
        elif ((temp['movement'] == 'downstairs').all()):
            tmove.append('downstairs')

        temp['combined'] = temp['y'] + temp['z']
        tdata.append(np.asarray(temp['combined']))

training_data = pd.DataFrame(tmove)
training_data['joined'] = pd.Series(tdata, index = training_data.index)
training_data

training_data.rename(columns={0: 'movement'}, inplace=True)
training_data = pd.concat([training_data['movement'], training_data.pop('joined').apply(pd.Series)], axis=1).dropna(axis=1)

training_data = training_data.iloc[324:,:].reset_index(drop = True)
training_data


,movement,0,1,2,3,4,5,6,7,8,...,83,84,85,86,87,88,89,90,91,92
0,walking,-6.299200,-7.129269,-7.689933,-7.941292,-7.872529,-7.499265,-6.853884,-5.972327,-4.883463,...,1.396235,-1.158375,-3.434326,-5.215625,-6.354383,-6.791119,-6.553549,-5.737138,-4.479456,-2.939246
1,walking,-5.826448,-6.552961,-7.071940,-7.356166,-7.397415,-7.203078,-6.788295,-6.166123,-5.340300,...,0.503542,-2.057842,-4.266476,-5.912852,-6.863920,-7.079308,-6.606717,-5.562094,-4.105350,-2.418699
2,walking,-5.648641,-6.209251,-6.599462,-6.803741,-6.819027,-6.652525,-6.315910,-5.817604,-5.156343,...,1.367233,-1.109564,-3.285103,-4.958047,-5.997780,-6.360420,-6.085328,-5.275470,-4.071965,-2.632211
3,walking,-5.119448,-5.927312,-6.566026,-7.002723,-7.221440,-7.222314,-7.016274,-6.615977,-6.026269,...,3.478748,1.083779,-1.255825,-3.312248,-4.892022,-5.865480,-6.184115,-5.880049,-5.049672,-3.830560
4,walking,-5.157422,-5.971403,-6.593845,-6.990992,-7.148891,-7.073522,-6.786384,-6.315513,-5.684347,...,5.818629,3.541135,1.075941,-1.352644,-3.518975,-5.227447,-6.341483,-6.801679,-6.626508,-5.896750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,downstairs,-0.988269,-2.122362,-3.128084,-3.948573,-4.541400,-4.881309,-4.961177,-4.790797,-4.394018,...,-4.624509,-5.016503,-5.083610,-4.808133,-4.190601,-3.255424,-2.054859,-0.669188,0.798198,2.230763
95,downstairs,0.609541,1.986818,3.379457,4.694758,5.826160,6.669487,7.142123,7.199415,6.842893,...,0.137284,1.418727,2.640271,3.707345,4.538441,5.074125,5.281929,5.156875,4.718271,4.004329
96,downstairs,-0.945662,-1.528726,-2.053644,-2.483161,-2.792156,-2.970517,-3.023860,-2.972105,-2.846118,...,1.285036,2.194872,3.052110,3.803471,4.394825,4.777308,4.913141,4.780414,4.376200,3.717431
97,downstairs,-1.829955,-2.778048,-3.557972,-4.126523,-4.457902,-4.544385,-4.394924,-4.031828,-3.486414,...,0.788891,-0.280010,-1.319224,-2.260254,-3.038236,-3.597293,-3.895373,-3.907862,-3.629609,-3.075485


In [46]:
training_data.loc[training_data['movement'] == 'walking'].count()

movement    33
0           33
1           33
2           33
3           33
            ..
88          33
89          33
90          33
91          33
92          33
Length: 94, dtype: int64

In [49]:
X = training_data.drop(columns = ['movement'])
y = training_data['movement']

X_train, X_valid, y_train, y_valid = train_test_split(X, y)

bayes_model = make_pipeline(
    StandardScaler(),
    GaussianNB()
)

knn_model = make_pipeline(
    StandardScaler(),
    KNeighborsClassifier(n_neighbors = 8)
)

nn_model = make_pipeline(
    StandardScaler(),
    MLPClassifier(solver = 'lbfgs', hidden_layer_sizes = (16,8,4), activation = 'logistic', max_iter=100000)
)

dt_model = make_pipeline(
    StandardScaler(),
    DecisionTreeClassifier(max_depth = 200)
)

rf_model = make_pipeline(
    RandomForestClassifier(n_estimators = 1500 , max_depth = 4, min_samples_leaf = 3)
)

en_model = make_pipeline(
    VotingClassifier([
        ('nb', GaussianNB()),
        ('knn', KNeighborsClassifier(8)),
        ('tree1', DecisionTreeClassifier(max_depth= 20)),
        ('tree2', DecisionTreeClassifier(min_samples_leaf=4)),
        ('rf', RandomForestClassifier(n_estimators = 250 , max_depth = 15, min_samples_leaf = 2)),
    ('mlp', MLPClassifier(solver = 'lbfgs', hidden_layer_sizes = (16,8,6), activation = 'logistic', max_iter=100000) )])
    
)

print('bayes')
bayes_model.fit(X_train, y_train)
print(bayes_model.score(X_train, y_train))
print(bayes_model.score(X_valid, y_valid))
y_predicted = bayes_model.predict(X_valid)
print(classification_report(y_valid, y_predicted))

print('')
print('knn')
knn_model.fit(X_train, y_train)
print(knn_model.score(X_train, y_train))
print(knn_model.score(X_valid, y_valid))
y_predicted = knn_model.predict(X_valid)
print(classification_report(y_valid, y_predicted))

print('')
print('neural net')
nn_model.fit(X_train, y_train)
print(nn_model.score(X_train, y_train))
print(nn_model.score(X_valid, y_valid))
y_predicted = nn_model.predict(X_valid)
print(classification_report(y_valid, y_predicted))

print('')
print('decision tree')
dt_model.fit(X_train, y_train)
print(dt_model.score(X_train, y_train))
print(dt_model.score(X_valid, y_valid))
y_predicted = dt_model.predict(X_valid)
print(classification_report(y_valid, y_predicted))

print('')
print('random forest')
rf_model.fit(X_train, y_train)
print(rf_model.score(X_train, y_train))
print(rf_model.score(X_valid, y_valid))
y_predicted = rf_model.predict(X_valid)
print(classification_report(y_valid, y_predicted))

print('')
print('ensemble')
en_model.fit(X_train, y_train)
print(en_model.score(X_train, y_train))
print(en_model.score(X_valid, y_valid))
y_predicted = en_model.predict(X_valid)
print(classification_report(y_valid, y_predicted))

bayes
0.8378378378378378
0.8
              precision    recall  f1-score   support

  downstairs       0.50      0.80      0.62         5
    upstairs       1.00      0.70      0.82        10
     walking       0.90      0.90      0.90        10

    accuracy                           0.80        25
   macro avg       0.80      0.80      0.78        25
weighted avg       0.86      0.80      0.81        25


knn
0.8783783783783784
0.72
              precision    recall  f1-score   support

  downstairs       0.38      0.60      0.46         5
    upstairs       0.80      0.80      0.80        10
     walking       1.00      0.70      0.82        10

    accuracy                           0.72        25
   macro avg       0.72      0.70      0.70        25
weighted avg       0.80      0.72      0.74        25


neural net
0.9864864864864865
0.8
              precision    recall  f1-score   support

  downstairs       0.62      1.00      0.77         5
    upstairs       0.80      0.80   

In [52]:
y_valid

0        walking
94    downstairs
72      upstairs
25       walking
46      upstairs
4        walking
30       walking
19       walking
14       walking
65    downstairs
9        walking
32       walking
79      upstairs
95    downstairs
40      upstairs
66    downstairs
6        walking
49      upstairs
35      upstairs
85    downstairs
70      upstairs
42      upstairs
50      upstairs
18       walking
37      upstairs
Name: movement, dtype: object